In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import cv2  
from tqdm import tqdm
from utils import show_image, main_color

from sklearn.cluster import KMeans
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76

In [2]:
train_path = "data/train"
test_path = "data/test"

train = os.listdir(train_path)
test = os.listdir(test_path)

In [4]:
train_image = [cv2.cvtColor(cv2.imread(os.path.join(train_path, image)), cv2.COLOR_BGR2RGB)  for image in train ]

In [30]:
trian_label = pd.read_csv("data/imagenames.csv")

color_total_train,weight_total_train = main_color(train_image[0:5],3,False)


100%|██████████| 5/5 [00:05<00:00,  1.07s/it]


In [10]:
trian_label = pd.read_csv("data/train.csv")

In [31]:
new_label = trian_label[:5]

In [32]:
new_label

,id
0,IMG4287_3
1,IMG4288_5
2,IMG4289_5
3,IMG4290_4
4,IMG4291_5


In [33]:
new_label["color"] = pd.Series(color_total_train)
new_label["weight"] = pd.Series(weight_total_train)

<ipython-input-33-9654ee1a5c12>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_label["color"] = pd.Series(color_total_train)
<ipython-input-33-9654ee1a5c12>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_label["weight"] = pd.Series(weight_total_train)


In [34]:
new_label

,id,color,weight
0,IMG4287_3,"[[58.18190220847422, 61.971327245673066, 26.59...","[0.6353916666666667, 0.28881666666666667, 0.07..."
1,IMG4288_5,"[[131.6306640991129, 131.84841771612656, 136.9...","[0.24418333333333334, 0.383675, 0.372141666666..."
2,IMG4289_5,"[[55.8263638360049, 56.119668308394964, 32.303...","[0.38769166666666666, 0.21459583333333335, 0.3..."
3,IMG4290_4,"[[119.37874918334911, 120.6083625348898, 122.0...","[0.14006666666666667, 0.80195, 0.0579833333333..."
4,IMG4291_5,"[[176.5967263383511, 180.22641643361325, 181.0...","[0.587025, 0.24821666666666667, 0.164758333333..."
